In [1]:
import pandas as pd

Q1

In [2]:
# 1
oil_prices = pd.DataFrame({
    'U.K. Brent': {'2013-Q1': 112.9, '2013-Q2': 103.0, 
                   '2013-Q3': 110.1, '2013-Q4': 109.4},
    'Dubai':      {'2013-Q1': 108.1, '2013-Q2': 100.8, 
                   '2013-Q3': 106.1, '2013-Q4': 106.7},
    'West Texas': {'2013-Q1': 94.4,  '2013-Q2': 94.2,  
                   '2013-Q3': 105.8, '2013-Q4': 97.4}})
oil_prices

,Dubai,U.K. Brent,West Texas
2013-Q1,108.1,112.9,94.4
2013-Q2,100.8,103.0,94.2
2013-Q3,106.1,110.1,105.8
2013-Q4,106.7,109.4,97.4


In [3]:
# 2
oil_prices.loc['2013-Q2':'2013-Q3', ['West Texas', 'U.K. Brent']]

,West Texas,U.K. Brent
2013-Q2,94.2,103.0
2013-Q3,105.8,110.1


In [4]:
# 4
oil_prices.loc[:, oil_prices.loc['2013-Q1'] > 110]

,U.K. Brent
2013-Q1,112.9
2013-Q2,103.0
2013-Q3,110.1
2013-Q4,109.4


In [5]:
# 5
oil_prices.iloc[0, 1:3]

U.K. Brent    112.9
West Texas     94.4
Name: 2013-Q1, dtype: float64

In [6]:
# 6
oil_prices.idxmax()

Dubai         2013-Q1
U.K. Brent    2013-Q1
West Texas    2013-Q3
dtype: object

In [7]:
# 7
oil_prices.unstack()

Dubai       2013-Q1    108.1
            2013-Q2    100.8
            2013-Q3    106.1
            2013-Q4    106.7
U.K. Brent  2013-Q1    112.9
            2013-Q2    103.0
            2013-Q3    110.1
            2013-Q4    109.4
West Texas  2013-Q1     94.4
            2013-Q2     94.2
            2013-Q3    105.8
            2013-Q4     97.4
dtype: float64

In [8]:
# 8
oil_prices.stack()

2013-Q1  Dubai         108.1
         U.K. Brent    112.9
         West Texas     94.4
2013-Q2  Dubai         100.8
         U.K. Brent    103.0
         West Texas     94.2
2013-Q3  Dubai         106.1
         U.K. Brent    110.1
         West Texas    105.8
2013-Q4  Dubai         106.7
         U.K. Brent    109.4
         West Texas     97.4
dtype: float64

In [9]:
# 9
oil_prices.reset_index().iloc[:, 0].str.split('-', expand=True)

,0,1
0,2013,Q1
1,2013,Q2
2,2013,Q3
3,2013,Q4


In [10]:
# 10
pd.concat([oil_prices, pd.DataFrame({
                        'Dubai':      {'2014-Q1': 107.3},
                        'U.K. Brent': {'2014-Q1': 102.5},
                        'Urals':      {'2014-Q1': 99.8}})])

,Dubai,U.K. Brent,Urals,West Texas
2013-Q1,108.1,112.9,NaN,94.4
2013-Q2,100.8,103.0,NaN,94.2
2013-Q3,106.1,110.1,NaN,105.8
2013-Q4,106.7,109.4,NaN,97.4
2014-Q1,107.3,102.5,99.8,NaN


Q2

In [24]:
flights = pd.read_csv('flights.csv')
flights.head()

,Month,Day,Weekday,Airline,Origin,Dest,AirTime,Distance,ArrivalDelay,Diverted,Cancelled
0,1,1,4,WN,LAX,SLC,94.0,590,65.0,0,0
1,1,1,4,UA,DEN,IAD,154.0,1452,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,85.0,641,35.0,0,0
3,1,1,4,AA,DFW,DCA,126.0,1192,-7.0,0,0
4,1,1,4,WN,LAX,MCI,166.0,1363,39.0,0,0


**1**. Find the month and day in which the longest arrival delay occurred.

In [12]:
flights.loc[flights['ArrivalDelay'].idxmax(), ['Month', 'Day']]
#flights.iloc[flights['ArrivalDelay'].argmax(), [0, 1]]

Month    12
Day      30
Name: 58188, dtype: object

**2**. Find the three airlines with the longest arrival delays.

In [35]:
#flights.loc[flights['ArrivalDelay'].nlargest(3).index, 'Airline']
#flights.sort_values('ArrivalDelay', ascending=False)['Airline'][:3]
#flights.groupby('Airline')['ArrivalDelay'].max().sort_values(ascending=False)[:3]
flights.groupby('Airline')['ArrivalDelay'].max()

Airline
AA     858.0
AS     344.0
B6     331.0
DL     741.0
EV     669.0
F9     839.0
HA     298.0
MQ     357.0
NK     474.0
OO     724.0
UA    1185.0
US     431.0
VX     236.0
WN     493.0
Name: ArrivalDelay, dtype: float64

**3**. Find the three airlines with the highest number of cancelled or diverted flights.

In [29]:
flights[(flights['Cancelled'] == 1) | (flights['Diverted'] == 1)]['Airline'].value_counts()[:3]

#flights['DivertedOrCancelled'] = flights['Diverted'] + flights['Cancelled']
#flights.groupby('Airline')['DivertedOrCancelled'].sum().nlargest(3)

AA    180
OO    163
EV    161
Name: Airline, dtype: int64

**4**. Find the airlines with more than 2% of cancelled flights. For each such airline, print its name
and the precentage of cancelled flights.

In [39]:
#flights.groupby('Airline').filter(lambda x: x['Cancelled'].mean() > 0.02) \
#       .groupby('Airline')['Cancelled'].mean()
pct_cancelled = flights.groupby('Airline')['Cancelled'].apply(lambda x: x.sum() / len(x))
pct_cancelled[pct_cancelled > 0.02]

Airline
EV    0.024923
MQ    0.043791
OO    0.021554
Name: Cancelled, dtype: float64

**5**. For each origin and destination, find the average and variance of the airtime.

In [16]:
flights.groupby(['Origin', 'Dest'])['AirTime'].agg(['mean', 'var']).head()

mean        var
Origin Dest                       
ATL    ABE    96.387097  45.778495
       ABQ   170.500000  87.866667
       ABY    28.578947   6.590643
       ACY    91.333333  11.466667
       AEX    78.725000  47.332692

**6**. Find the airline that makes the most flights between 500 and 1,000 miles.

In [17]:
flights[flights['Distance'].between(500, 1000)].groupby('Airline').size().idxmax()

'DL'

**7**. Create a bar plot showing the number of flights originating from each city.

In [18]:
df = flights.groupby('Origin').size()
df.plot.bar();

**8**. Change the Weekday column to have the day name instead of day number, e.g. 1 - Sun, 2 - Mon, etc.

In [19]:
week_day = { 1: 'SUN', 2: 'Mon', 3: 'Tue', 4: 'Wed', 5: 'Thu', 6: 'Fri', 7: 'Sat'}
flights['Weekday'] = flights['Weekday'].map(week_day)
#flights['Weekday'] = flights['Weekday'].apply(lambda x: week_day[x])
flights.head()

,Month,Day,Weekday,Airline,Origin,Dest,AirTime,Distance,ArrivalDelay,Diverted,Cancelled
0,1,1,Wed,WN,LAX,SLC,94.0,590,65.0,0,0
1,1,1,Wed,UA,DEN,IAD,154.0,1452,-13.0,0,0
2,1,1,Wed,MQ,DFW,VPS,85.0,641,35.0,0,0
3,1,1,Wed,AA,DFW,DCA,126.0,1192,-7.0,0,0
4,1,1,Wed,WN,LAX,MCI,166.0,1363,39.0,0,0


**9**. Join the Month and Day columns into a single column named Date. For example, if month=3, day=15, the date column should have 3/15.

In [27]:
#flights['Date'] = flights['Month'].astype(str).str.cat(flights['Day'].astype(str), sep='/')
#flights.drop(['Month', 'Day'], axis=1, inplace=True)
flights['Date'] = flights['Month'].map(str) + '/' + flights['Day'].map(str)
flights.head()

,Month,Day,Weekday,Airline,Origin,Dest,AirTime,Distance,ArrivalDelay,Diverted,Cancelled,Date
0,1,1,4,WN,LAX,SLC,94.0,590,65.0,0,0,1/1
1,1,1,4,UA,DEN,IAD,154.0,1452,-13.0,0,0,1/1
2,1,1,4,MQ,DFW,VPS,85.0,641,35.0,0,0,1/1
3,1,1,4,AA,DFW,DCA,126.0,1192,-7.0,0,0,1/1
4,1,1,4,WN,LAX,MCI,166.0,1363,39.0,0,0,1/1


**10**. Find the longest sequence of on-time flights per airline (an on-time flight is a flight with less than 15 minutes arrival delay).

In [21]:
def max_streak(s):
    return (s >= 15).cumsum().value_counts().max() - 1
flights.groupby('Airline')['ArrivalDelay'].agg(max_streak)

Airline
AA    50.0
AS    48.0
B6    25.0
DL    91.0
EV    42.0
F9    22.0
HA    17.0
MQ    51.0
NK    24.0
OO    36.0
UA    46.0
US    44.0
VX    38.0
WN    43.0
Name: ArrivalDelay, dtype: float64

In [22]:
# flights['OnTime'] = (flights['ArrivalDelay'] < 15) * 1

# def max_streak(s):
#    s1 = s.cumsum()
#    return s.mul(s1).diff().where(lambda x: x < 0) \
#           .ffill().add(s1, fill_value=0).max()
    
# flights.groupby('Airline')['OnTime'].agg(max_streak)